In [327]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
pd.options.display.max_rows=300
submission=pd.read_csv(r'C:\Users\enesm\OneDrive\Masaüstü\KAGGLE\Iyzico\Fresh Start\submission_first_stage.csv')
final_df=pd.read_csv(r'C:\Users\enesm\OneDrive\Masaüstü\KAGGLE\Iyzico\Fresh Start\final_df_first_stage.csv')

In [328]:
class Lessthen12monthmerchants:
    def __init__(self,data,submission):
        self.data=data
        self.submission=submission
        self.merchants_without_corresponding = []


    def split_the_merchants(self,category):

        self.category=category
        self.less12merchants=[]
        self.more12merchants=[]

        # get the merchants in to a list for that category
        subcat_df= self.data[self.data['category']==self.category]
        unique_merchants = subcat_df['merchant_id'].unique()
        merchant_dfs = {merchant: subcat_df[subcat_df['merchant_id'] == merchant] for merchant in unique_merchants}

        # Get unique merchants
        for j in unique_merchants:
            if np.median(merchant_dfs[j]['active_months'].values) <=12 :
                self.less12merchants.append(j)
            else:
                self.more12merchants.append(j)

        return self.less12merchants,self.more12merchants


    def find_corresponding_merchant(self, less12merchant_id):
        if self.more12merchants:
            return self.more12merchants[0]
        else:
            self.merchants_without_corresponding.append(less12merchant_id)  # Add to the list
            return None

    def forecast_less12_merchants(self):
        forecasts = []


        for less12merchant in self.less12merchants:
            corresponding_merchant = self.find_corresponding_merchant(less12merchant)
            if corresponding_merchant is None:
                continue

            less12_df = self.data[self.data['merchant_id'] == less12merchant]
            more12_df = self.data[self.data['merchant_id'] == corresponding_merchant]
            submission_df=self.submission[self.submission['merchant_id']==corresponding_merchant]

            # Find overlapping dates
            overlapping_dates = less12_df[less12_df['date'].isin(more12_df['date'])]
            ratios = []

            for date in overlapping_dates['date']:
                less12_value = less12_df[less12_df['date'] == date]['net_payment_count'].values[0]
                more12_value = more12_df[more12_df['date'] == date]['net_payment_count'].values[0]
                if (more12_value != 0 )& (less12_value>0):
                    ratios.append(less12_value / more12_value)

            if ratios:
                avg_ratio = np.mean(ratios)
                forecast_dates=less12_df.loc[less12_df.index[-3]:]
                
                for index, row in forecast_dates.iterrows():
                    forecast_date = row['date']
                    if forecast_date in more12_df['date'].values:
                        more12_forecast = submission_df[submission_df['date'] == forecast_date]['net_payment_count'].values[0]
                        if not pd.isna(more12_forecast):
                            forecast_value = avg_ratio * more12_forecast
                            forecasts.append((less12merchant, forecast_date, forecast_value))

        return forecasts

   


    def run(self, category):
        # Step 1: Split the merchants based on the specified category
        self.split_the_merchants(category)

        # Step 2: Forecast sales for merchants with less than 12 months of data
        forecasts = self.forecast_less12_merchants()

        # Step 3: Process and return the submitted dataframe
        submitted_df = self.submission.copy()
        for merchant_id, forecast_date, forecast_value in forecasts:
            # Update the submission dataframe with the forecasted values
            condition = (submitted_df['merchant_id'] == merchant_id) & (submitted_df['date'] == forecast_date)
            submitted_df.loc[condition, 'net_payment_count'] = forecast_value

        return submitted_df


In [311]:
categories=set(final_df.category)
from tqdm import tqdm
all_merchants_without_corresponding = []
for category in tqdm(categories):
    processor = Lessthen12monthmerchants(final_df, submission)
    submission = processor.run(category=category)
    all_merchants_without_corresponding.extend(processor.merchants_without_corresponding)

# Now all_merchants_without_corresponding contains data across all categories
print("Merchants without corresponding merchants:", all_merchants_without_corresponding)

  3%|▎         | 28/963 [00:01<01:03, 14.71it/s]


KeyboardInterrupt: 

In [246]:
submission[submission.merchant_id=='merchant_10025']

,category,merchant_id,date,id,month_id,net_payment_count
33,85,merchant_10025,2023-10-01,202310merchant_10025,2023-10-01,2.789909
34,85,merchant_10025,2023-11-01,202311merchant_10025,2023-11-01,3.189312
35,85,merchant_10025,2023-12-01,202312merchant_10025,2023-12-01,3.576216


# DİĞER KISIM

In [329]:
final_df.date=pd.to_datetime(final_df.date)

In [330]:
# Specify the dates you want to remove
dates_to_remove = ['2023-10-01', '2023-11-01', '2023-12-01']

# Filter the DataFrame
temp_df = final_df[~final_df['date'].isin(pd.to_datetime(dates_to_remove))]


In [331]:
temp_df

,date,merchant_id,net_payment_count,is_train,category,flag,active_months,max_consecutive
0,2022-05-01,merchant_10000,3.0,True,245.0,1.0,11.0,3.0
1,2022-06-01,merchant_10000,4.0,True,245.0,1.0,11.0,3.0
2,2022-07-01,merchant_10000,0.0,True,245.0,1.0,11.0,3.0
3,2022-08-01,merchant_10000,0.0,True,245.0,1.0,11.0,3.0
4,2022-09-01,merchant_10000,0.0,True,245.0,1.0,11.0,3.0
...,...,...,...,...,...,...,...,...
206157,2023-05-01,merchant_9991,7.0,True,476.0,1.0,16.0,10.0
206158,2023-06-01,merchant_9991,10.0,True,476.0,1.0,16.0,10.0
206159,2023-07-01,merchant_9991,15.0,True,476.0,1.0,16.0,10.0
206160,2023-08-01,merchant_9991,9.0,True,476.0,1.0,16.0,10.0


In [332]:
temp_df

,date,merchant_id,net_payment_count,is_train,category,flag,active_months,max_consecutive
0,2022-05-01,merchant_10000,3.0,True,245.0,1.0,11.0,3.0
1,2022-06-01,merchant_10000,4.0,True,245.0,1.0,11.0,3.0
2,2022-07-01,merchant_10000,0.0,True,245.0,1.0,11.0,3.0
3,2022-08-01,merchant_10000,0.0,True,245.0,1.0,11.0,3.0
4,2022-09-01,merchant_10000,0.0,True,245.0,1.0,11.0,3.0
...,...,...,...,...,...,...,...,...
206157,2023-05-01,merchant_9991,7.0,True,476.0,1.0,16.0,10.0
206158,2023-06-01,merchant_9991,10.0,True,476.0,1.0,16.0,10.0
206159,2023-07-01,merchant_9991,15.0,True,476.0,1.0,16.0,10.0
206160,2023-08-01,merchant_9991,9.0,True,476.0,1.0,16.0,10.0


In [333]:
# Assuming merch_dict is correctly computed
merch_dict = temp_df.groupby('merchant_id')['net_payment_count'].mean().to_dict()


In [334]:

submission['net_payment_count_mapped'] = submission['merchant_id'].map(merch_dict)


In [336]:
submission['net_payment_count'] = submission['net_payment_count'].fillna(submission['net_payment_count_mapped'])

In [325]:
submission

,category,merchant_id,date,id,month_id,net_payment_count,net_payment_count_mapped
0,727,merchant_1,2023-10-01,202310merchant_1,2023-10-01,0.000000,NaN
1,727,merchant_1,2023-11-01,202311merchant_1,2023-11-01,0.000000,NaN
2,727,merchant_1,2023-12-01,202312merchant_1,2023-12-01,0.000000,NaN
3,180,merchant_10,2023-10-01,202310merchant_10,2023-10-01,0.000000,NaN
4,180,merchant_10,2023-11-01,202311merchant_10,2023-11-01,0.000000,NaN
...,...,...,...,...,...,...,...
78175,476,merchant_9991,2023-11-01,202311merchant_9991,2023-11-01,6.953849,9.411765
78176,476,merchant_9991,2023-12-01,202312merchant_9991,2023-12-01,21.397732,9.411765
78177,799,merchant_9993,2023-10-01,202310merchant_9993,2023-10-01,0.000000,NaN
78178,799,merchant_9993,2023-11-01,202311merchant_9993,2023-11-01,0.000000,NaN


In [335]:
submission.isna().sum()

category                        0
merchant_id                     0
date                            0
id                              0
month_id                        0
net_payment_count            1989
net_payment_count_mapped    56319
dtype: int64

In [338]:
submission=submission.drop(columns=['category',"merchant_id","date","month_id",'net_payment_count_mapped'],axis=1)

In [339]:
submission.to_csv('ücüncü_deneme.csv',index=False)

In [340]:
submission

,id,net_payment_count
0,202310merchant_1,0.000000
1,202311merchant_1,0.000000
2,202312merchant_1,0.000000
3,202310merchant_10,0.000000
4,202311merchant_10,0.000000
...,...,...
78175,202311merchant_9991,6.953849
78176,202312merchant_9991,21.397732
78177,202310merchant_9993,0.000000
78178,202311merchant_9993,0.000000


In [284]:
submission.sort_values(by='id').to_csv('karşılaştırkaggleile.csv',index=False)

In [287]:
submission.net_payment_count.isna().sum()

3

In [288]:
submission.query("net_payment_count.isna()")

,id,net_payment_count
4242,202310merchant_13267,NaN
4243,202311merchant_13267,NaN
4244,202312merchant_13267,NaN


In [337]:
submission

,category,merchant_id,date,id,month_id,net_payment_count,net_payment_count_mapped
0,727,merchant_1,2023-10-01,202310merchant_1,2023-10-01,0.000000,NaN
1,727,merchant_1,2023-11-01,202311merchant_1,2023-11-01,0.000000,NaN
2,727,merchant_1,2023-12-01,202312merchant_1,2023-12-01,0.000000,NaN
3,180,merchant_10,2023-10-01,202310merchant_10,2023-10-01,0.000000,NaN
4,180,merchant_10,2023-11-01,202311merchant_10,2023-11-01,0.000000,NaN
...,...,...,...,...,...,...,...
78175,476,merchant_9991,2023-11-01,202311merchant_9991,2023-11-01,6.953849,9.411765
78176,476,merchant_9991,2023-12-01,202312merchant_9991,2023-12-01,21.397732,9.411765
78177,799,merchant_9993,2023-10-01,202310merchant_9993,2023-10-01,0.000000,NaN
78178,799,merchant_9993,2023-11-01,202311merchant_9993,2023-11-01,0.000000,NaN
